# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [1]:
from keras.layers import GRU, Input, Dense, TimeDistributed, SimpleRNN, LSTM
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.callbacks import ModelCheckpoint
from keras.layers import Dropout
from keras.layers import Dropout, Bidirectional, RepeatVector
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
import pandas as pd
import nltk
df = pd.read_csv('./data/jokes.csv',sep=',',nrows=1000)

In [3]:
#english_sentences = [word.lower() for sent in df.Question.values for word in nltk.tokenize.word_tokenize(sent)]
#french_sentences = [' '.join(word.lower()) for sent in df.Answer.values for word in nltk.tokenize.word_tokenize(sent)]
english_sentences = df.Question.values
french_sentences = df.Answer.values

print('Dataset Loaded')

Dataset Loaded


In [4]:
len(french_sentences)

1000

### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  Did you hear about the Native American man that drank 200 cups of tea?
small_vocab_fr Line 1:  He nearly drown in his own tea pee.
small_vocab_en Line 2:  What's the best anti diarrheal prescription?
small_vocab_fr Line 2:  Mycheexarphlexin


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
import collections


english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

8850 English words.
2706 unique English words.
10 Most common words in the English dataset:
"a" "the" "What" "do" "you" "Why" "to" "call" "and" "did"

6173 French words.
2763 unique French words.
10 Most common words in the French dataset:
"the" "a" "to" "A" "Because" "I" "and" "it" "you" "in"


In [7]:
english_words_counter.most_common(20)

[('a', 471),
 ('the', 450),
 ('What', 403),
 ('do', 312),
 ('you', 310),
 ('Why', 189),
 ('to', 152),
 ('call', 147),
 ('and', 143),
 ('did', 140),
 ('in', 128),
 ("What's", 105),
 ('How', 101),
 ('is', 96),
 ('of', 95),
 ('does', 85),
 ('say', 75),
 ('between', 70),
 ('difference', 64),
 ('when', 61)]

For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [8]:
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    
    tokenizer = Tokenizer(num_words=None, filters="", lower=False, split=" ")
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer


# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'.': 1, 'quick': 2, 'a': 3, 'The': 4, 'brown': 5, 'fox': 6, 'jumps': 7, 'over': 8, 'the': 9, 'lazy': 10, 'dog': 11, 'By': 12, 'Jove': 13, ',': 14, 'my': 15, 'study': 16, 'of': 17, 'lexicography': 18, 'won': 19, 'prize': 20, 'This': 21, 'is': 22, 'short': 23, 'sentence': 24}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [4, 2, 5, 6, 7, 8, 9, 10, 11, 1]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [12, 13, 14, 15, 2, 16, 17, 18, 19, 3, 20, 1]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [21, 22, 3, 23, 24, 1]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=15):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length==None:
        maxLenX = 0
        for sequence in x:
            if len(sequence) > maxLenX:
                maxLenX = len(sequence)
        
        padded = pad_sequences(sequences=x,maxlen=maxLenX, dtype='int32', padding='post', truncating='post', value=0)

    else:
        padded = pad_sequences(sequences=x,maxlen=length, dtype='int32', padding='post', truncating='post', value=0)
    return padded


# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 4  2  5  6  7  8  9 10 11  1]
  Output: [ 4  2  5  6  7  8  9 10 11  1  0  0  0  0  0]
Sequence 2 in x
  Input:  [12 13 14 15  2 16 17 18 19  3 20  1]
  Output: [12 13 14 15  2 16 17 18 19  3 20  1  0  0  0]
Sequence 3 in x
  Input:  [21 22  3 23 24  1]
  Output: [21 22  3 23 24  1  0  0  0  0  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    print(preprocess_y.shape)
    
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')


(1000, 15)
Data Preprocessed


In [11]:
# Embedding matrix

#word embeddings from glove

embeddings_index = dict()

with open('./data/glove.6B.100d.txt',encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word]= coefs

print('loaded %s word vectors' % len(embeddings_index))

loaded 400000 word vectors


In [12]:
#encoder matrix
num_english_tokens = len(english_tokenizer.word_index)+1

encoder_embedding_matrix = np.zeros((num_english_tokens, 100))

for word,i in english_tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        encoder_embedding_matrix[i] = embedding_vector

In [13]:
encoder_embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.27085999,  0.044006  , -0.02026   , ..., -0.4923    ,
         0.63687003,  0.23642001],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       ..., 
       [ 0.061859  ,  0.10992   ,  0.27384001, ..., -0.20844001,
         0.10268   , -0.17731   ],
       [ 0.38402   ,  0.11894   , -0.11175   , ...,  0.20793   ,
         0.37232   , -0.011673  ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [14]:
learning_rate = 0.01
hidden_dim = 256
embedding_vector_length = 100

In [15]:

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))


In [16]:
input_shape =tmp_x.shape
output_sequence_length =preproc_french_sentences.shape[1]
english_vocab_size =len(english_tokenizer.word_index)+1
french_vocab_size=len(french_tokenizer.word_index)+1


In [57]:
    #Variation - 3 Hidden LSTM Layer and 2 Dense ouput layer with higher Dropout Rate

model = Sequential()
model.add(Embedding(english_vocab_size, embedding_vector_length,weights=[encoder_embedding_matrix], trainable= False,
                    input_shape= input_shape[1:]))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Dense(french_vocab_size))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(french_vocab_size))
model.add(Activation('softmax'))
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(learning_rate),
              metrics=['accuracy'])
print(model.summary())


model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=500, validation_split=0.2)

model.save('./saved_models/final_rnn_model6_embed.hdf5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 15, 100)           270700    
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 15, 512)           731136    
_________________________________________________________________
dropout_13 (Dropout)         (None, 15, 512)           0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 15, 512)           1574912   
_________________________________________________________________
dropout_14 (Dropout)         (None, 15, 512)           0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 15, 512)           1574912   
_________________________________________________________________
dropout_15 (Dropout)         (None, 15, 512)           0         
__________

Epoch 18/500
800/800 [==============================] - ETA: 32s - loss: 6.1537 - acc: 0.61 - ETA: 27s - loss: 5.9313 - acc: 0.63 - ETA: 20s - loss: 6.0026 - acc: 0.62 - ETA: 14s - loss: 6.0551 - acc: 0.62 - ETA: 7s - loss: 6.0765 - acc: 0.6230 - ETA: 1s - loss: 6.0264 - acc: 0.626 - 43s 54ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 19/500
800/800 [==============================] - ETA: 30s - loss: 6.2377 - acc: 0.61 - ETA: 26s - loss: 6.1957 - acc: 0.61 - ETA: 20s - loss: 5.9047 - acc: 0.63 - ETA: 14s - loss: 5.9019 - acc: 0.63 - ETA: 7s - loss: 5.9741 - acc: 0.6294 - ETA: 1s - loss: 6.0278 - acc: 0.626 - 43s 53ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 20/500
800/800 [==============================] - ETA: 34s - loss: 6.3300 - acc: 0.60 - ETA: 27s - loss: 5.9607 - acc: 0.63 - ETA: 20s - loss: 6.0614 - acc: 0.62 - ETA: 14s - loss: 6.0425 - acc: 0.62 - ETA: 7s - loss: 6.0043 - acc: 0.6275 - ETA: 1s - loss: 5.9831 

800/800 [==============================] - ETA: 32s - loss: 5.6417 - acc: 0.65 - ETA: 27s - loss: 5.8809 - acc: 0.63 - ETA: 20s - loss: 5.8991 - acc: 0.63 - ETA: 14s - loss: 5.9942 - acc: 0.62 - ETA: 8s - loss: 5.9338 - acc: 0.6319 - ETA: 1s - loss: 5.9831 - acc: 0.628 - 44s 55ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 41/500
800/800 [==============================] - ETA: 31s - loss: 6.2797 - acc: 0.61 - ETA: 25s - loss: 6.1831 - acc: 0.61 - ETA: 20s - loss: 5.9159 - acc: 0.63 - ETA: 14s - loss: 6.0194 - acc: 0.62 - ETA: 7s - loss: 6.0278 - acc: 0.6260 - ETA: 1s - loss: 6.0208 - acc: 0.626 - 43s 53ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 42/500
800/800 [==============================] - ETA: 34s - loss: 6.2629 - acc: 0.61 - ETA: 27s - loss: 5.9942 - acc: 0.62 - ETA: 21s - loss: 6.0110 - acc: 0.62 - ETA: 14s - loss: 6.0047 - acc: 0.62 - ETA: 8s - loss: 5.9825 - acc: 0.6289 - ETA: 1s - loss: 6.0138 - acc: 0.626 

800/800 [==============================] - ETA: 32s - loss: 6.0614 - acc: 0.62 - ETA: 26s - loss: 6.2041 - acc: 0.61 - ETA: 20s - loss: 5.9747 - acc: 0.62 - ETA: 14s - loss: 5.9649 - acc: 0.62 - ETA: 7s - loss: 5.9691 - acc: 0.6297 - ETA: 1s - loss: 6.0278 - acc: 0.626 - 43s 54ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 63/500
800/800 [==============================] - ETA: 35s - loss: 6.3804 - acc: 0.60 - ETA: 26s - loss: 6.1370 - acc: 0.61 - ETA: 20s - loss: 6.0894 - acc: 0.62 - ETA: 14s - loss: 6.1621 - acc: 0.61 - ETA: 7s - loss: 6.1773 - acc: 0.6168 - ETA: 1s - loss: 6.0502 - acc: 0.624 - 43s 54ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 64/500
800/800 [==============================] - ETA: 33s - loss: 6.2293 - acc: 0.61 - ETA: 26s - loss: 6.0026 - acc: 0.62 - ETA: 20s - loss: 5.8543 - acc: 0.63 - ETA: 14s - loss: 5.8515 - acc: 0.63 - ETA: 7s - loss: 5.8331 - acc: 0.6381 - ETA: 1s - loss: 5.9341 - acc: 0.631 

800/800 [==============================] - ETA: 34s - loss: 6.6658 - acc: 0.58 - ETA: 27s - loss: 6.3091 - acc: 0.60 - ETA: 20s - loss: 6.0978 - acc: 0.62 - ETA: 14s - loss: 6.0908 - acc: 0.62 - ETA: 7s - loss: 6.1135 - acc: 0.6207 - ETA: 1s - loss: 6.0488 - acc: 0.624 - 43s 54ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 85/500
800/800 [==============================] - ETA: 32s - loss: 5.6249 - acc: 0.65 - ETA: 27s - loss: 6.0446 - acc: 0.62 - ETA: 20s - loss: 6.1174 - acc: 0.62 - ETA: 14s - loss: 6.0866 - acc: 0.62 - ETA: 7s - loss: 6.0832 - acc: 0.6226 - ETA: 1s - loss: 6.0264 - acc: 0.626 - 43s 53ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 86/500
800/800 [==============================] - ETA: 32s - loss: 5.9439 - acc: 0.63 - ETA: 26s - loss: 5.7802 - acc: 0.64 - ETA: 20s - loss: 5.7480 - acc: 0.64 - ETA: 14s - loss: 5.8536 - acc: 0.63 - ETA: 7s - loss: 5.9623 - acc: 0.6301 - ETA: 1s - loss: 5.9523 - acc: 0.630 

...

800/800 [==============================] - ETA: 33s - loss: 5.6417 - acc: 0.65 - ETA: 26s - loss: 6.0992 - acc: 0.62 - ETA: 21s - loss: 5.9887 - acc: 0.62 - ETA: 14s - loss: 5.9376 - acc: 0.63 - ETA: 8s - loss: 5.9892 - acc: 0.6284 - ETA: 1s - loss: 6.0152 - acc: 0.626 - 44s 54ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 393/500
800/800 [==============================] - ETA: 32s - loss: 5.8096 - acc: 0.63 - ETA: 27s - loss: 5.8389 - acc: 0.63 - ETA: 20s - loss: 6.0278 - acc: 0.62 - ETA: 14s - loss: 5.9607 - acc: 0.63 - ETA: 7s - loss: 6.0161 - acc: 0.6268 - ETA: 1s - loss: 6.0264 - acc: 0.626 - 43s 54ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 394/500
800/800 [==============================] - ETA: 33s - loss: 6.1202 - acc: 0.62 - ETA: 27s - loss: 6.2965 - acc: 0.60 - ETA: 20s - loss: 6.2321 - acc: 0.61 - ETA: 14s - loss: 6.1349 - acc: 0.61 - ETA: 8s - loss: 6.0379 - acc: 0.6254 - ETA: 1s - loss: 6.0460 - acc: 0.62

800/800 [==============================] - ETA: 32s - loss: 6.0698 - acc: 0.62 - ETA: 27s - loss: 5.9397 - acc: 0.63 - ETA: 20s - loss: 5.9579 - acc: 0.63 - ETA: 14s - loss: 5.9166 - acc: 0.63 - ETA: 7s - loss: 5.9724 - acc: 0.6295 - ETA: 1s - loss: 5.9845 - acc: 0.628 - 44s 55ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 415/500
800/800 [==============================] - ETA: 31s - loss: 5.8935 - acc: 0.63 - ETA: 26s - loss: 5.9607 - acc: 0.63 - ETA: 20s - loss: 5.8263 - acc: 0.63 - ETA: 14s - loss: 5.9649 - acc: 0.62 - ETA: 7s - loss: 6.0295 - acc: 0.6259 - ETA: 1s - loss: 6.0068 - acc: 0.627 - 43s 54ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 416/500
800/800 [==============================] - ETA: 33s - loss: 6.0614 - acc: 0.62 - ETA: 27s - loss: 6.1537 - acc: 0.61 - ETA: 20s - loss: 6.2097 - acc: 0.61 - ETA: 14s - loss: 6.0299 - acc: 0.62 - ETA: 7s - loss: 6.0178 - acc: 0.6267 - ETA: 1s - loss: 6.0040 - acc: 0.62

800/800 [==============================] - ETA: 35s - loss: 5.9103 - acc: 0.63 - ETA: 27s - loss: 6.0320 - acc: 0.62 - ETA: 21s - loss: 5.9691 - acc: 0.62 - ETA: 14s - loss: 5.9775 - acc: 0.62 - ETA: 8s - loss: 5.9456 - acc: 0.6311 - ETA: 1s - loss: 5.9803 - acc: 0.629 - 44s 55ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 437/500
800/800 [==============================] - ETA: 33s - loss: 5.9355 - acc: 0.63 - ETA: 26s - loss: 5.9775 - acc: 0.62 - ETA: 20s - loss: 5.9383 - acc: 0.63 - ETA: 14s - loss: 5.9796 - acc: 0.62 - ETA: 7s - loss: 5.8465 - acc: 0.6373 - ETA: 1s - loss: 5.9845 - acc: 0.628 - 43s 54ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 438/500
800/800 [==============================] - ETA: 33s - loss: 5.8264 - acc: 0.63 - ETA: 26s - loss: 5.8683 - acc: 0.63 - ETA: 20s - loss: 5.9859 - acc: 0.62 - ETA: 14s - loss: 5.9775 - acc: 0.62 - ETA: 7s - loss: 6.0530 - acc: 0.6245 - ETA: 1s - loss: 5.9873 - acc: 0.62

800/800 [==============================] - ETA: 32s - loss: 6.4308 - acc: 0.60 - ETA: 27s - loss: 5.8389 - acc: 0.63 - ETA: 20s - loss: 5.7816 - acc: 0.64 - ETA: 14s - loss: 5.7466 - acc: 0.64 - ETA: 8s - loss: 5.9556 - acc: 0.6305 - ETA: 1s - loss: 6.0334 - acc: 0.625 - 45s 56ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 459/500
800/800 [==============================] - ETA: 32s - loss: 6.5399 - acc: 0.59 - ETA: 26s - loss: 5.8054 - acc: 0.63 - ETA: 20s - loss: 5.7956 - acc: 0.64 - ETA: 14s - loss: 5.9586 - acc: 0.63 - ETA: 7s - loss: 5.9623 - acc: 0.6301 - ETA: 1s - loss: 5.9998 - acc: 0.627 - 43s 54ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 460/500
800/800 [==============================] - ETA: 33s - loss: 5.6333 - acc: 0.65 - ETA: 26s - loss: 5.8305 - acc: 0.63 - ETA: 20s - loss: 5.8263 - acc: 0.63 - ETA: 14s - loss: 5.8788 - acc: 0.63 - ETA: 7s - loss: 6.0278 - acc: 0.6260 - ETA: 1s - loss: 6.0320 - acc: 0.62

800/800 [==============================] - ETA: 31s - loss: 5.7340 - acc: 0.64 - ETA: 25s - loss: 5.5535 - acc: 0.65 - ETA: 20s - loss: 5.6696 - acc: 0.64 - ETA: 13s - loss: 5.9649 - acc: 0.62 - ETA: 7s - loss: 5.8902 - acc: 0.6346 - ETA: 1s - loss: 5.9117 - acc: 0.633 - 42s 53ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 481/500
800/800 [==============================] - ETA: 36s - loss: 5.9355 - acc: 0.63 - ETA: 28s - loss: 5.7550 - acc: 0.64 - ETA: 21s - loss: 5.8375 - acc: 0.63 - ETA: 14s - loss: 5.8620 - acc: 0.63 - ETA: 8s - loss: 5.9758 - acc: 0.6293 - ETA: 1s - loss: 5.9998 - acc: 0.627 - 44s 54ms/step - loss: 6.0030 - acc: 0.6276 - val_loss: 5.9264 - val_acc: 0.6323
Epoch 482/500
800/800 [==============================] - ETA: 33s - loss: 5.8012 - acc: 0.64 - ETA: 27s - loss: 5.9691 - acc: 0.62 - ETA: 20s - loss: 5.9579 - acc: 0.63 - ETA: 14s - loss: 6.0110 - acc: 0.62 - ETA: 7s - loss: 6.1000 - acc: 0.6216 - ETA: 1s - loss: 5.9887 - acc: 0.62

## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [17]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [18]:
def preprocess_test(raw_word, name, english_tokenizer):
    l1 = ['won’t','won\'t','wouldn’t','wouldn\'t','’m', '’re', '’ve', '’ll', '’s','’d', 'n’t', '\'m', '\'re', '\'ve', '\'ll', '\'s', '\'d', 'can\'t', 'n\'t', 'B: ', 'A: ', ',', ';', '.', '?', '!', ':', '. ?', ',   .', '. ,', 'EOS', 'BOS', 'eos', 'bos']
    l2 = ['will not','will not','would not','would not',' am', ' are', ' have', ' will', ' is', ' had', ' not', ' am', ' are', ' have', ' will', ' is', ' had', 'can not', ' not', '', '', ' ,', ' ;', ' .', ' ?', ' !', ' :', '? ', '.', ',', '', '', '', '']
    l3 = ['-', '_', ' *', ' /', '* ', '/ ', '\"', ' \\"', '\\ ', '--', '...', '. . .']
    l4 = ['jeffrey','fred','benjamin','paula','walter','rachel','andy','helen','harrington','kathy','ronnie','carl','annie','cole','ike','milo','cole','rick','johnny','loretta','cornelius','claire','romeo','casey','johnson','rudy','stanzi','cosgrove','wolfi','kevin','paulie','cindy','paulie','enzo','mikey','i\97','davis','jeffrey','norman','johnson','dolores','tom','brian','bruce','john','laurie','stella','dignan','elaine','jack','christ','george','frank','mary','amon','david','tom','joe','paul','sam','charlie','bob','marry','walter','james','jimmy','michael','rose','jim','peter','nick','eddie','johnny','jake','ted','mike','billy','louis','ed','jerry','alex','charles','tommy','bobby','betty','sid','dave','jeffrey','jeff','marty','richard','otis','gale','fred','bill','jones','smith','mickey']    

    raw_word = raw_word.lower()
    raw_word = raw_word.replace(', ' + name_of_computer, '')
    raw_word = raw_word.replace(name_of_computer + ' ,', '')

    for j, term in enumerate(l1):
        raw_word = raw_word.replace(term,l2[j])
        
    for term in l3:
        raw_word = raw_word.replace(term,' ')
    
    for term in l4:
        raw_word = raw_word.replace(', ' + term, ', ' + name)
        raw_word = raw_word.replace(' ' + term + ' ,' ,' ' + name + ' ,')
        raw_word = raw_word.replace('i am ' + term, 'i am ' + name_of_computer)
        raw_word = raw_word.replace('my name is' + term, 'my name is ' + name_of_computer)
    
    for j in range(30):
        raw_word = raw_word.replace('. .', '')
        raw_word = raw_word.replace('.  .', '')
        raw_word = raw_word.replace('..', '')
       
    for j in range(5):
        raw_word = raw_word.replace('  ', ' ')
        
    if raw_word[-1] !=  '!' and raw_word[-1] != '?' and raw_word[-1] != '.' and raw_word[-2:] !=  '! ' and raw_word[-2:] != '? ' and raw_word[-2:] != '. ':
        raw_word = raw_word + ' .'
    
    if raw_word == ' !' or raw_word == ' ?' or raw_word == ' .' or raw_word == ' ! ' or raw_word == ' ? ' or raw_word == ' . ':
        raw_word = 'what ?'
       
    
    if raw_word == '  .' or raw_word == ' .' or raw_word == '  . ':
        raw_word = 'i do not want to talk about it .'
    
    
    raw_new_string = []
    
    for word in nltk.tokenize.word_tokenize(raw_word):
        if word not in english_tokenizer.word_index.keys():
            word = name
            raw_new_string.append(word)
        else:
            raw_new_string.append(word)
            
    
    
            #for dict_word in english_tokenizer.word_index.keys():
            #    nlp_words = nlp(word)
            #    dict_tokens = nlp(dict_word)
            #    nlp_words.similarity(dict_tokens)
    
    
    return ' '.join(raw_new_string)


In [23]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model


que = ''
last_query  = ' '
last_last_query = ''
text = ' '
last_text = ''
name_of_computer = 'egg'

def final_predictions(model, x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    
    #model = load_model('saved_models/final_rnn_model6.hdf5')
    #model.summary()
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'
    

    que = ''

    print('computer: hi ! please type your name.\n')
    name = input('user: ')
    print('computer: hi , ' + name +' ! My name is ' + name_of_computer + '.\n') 

    
    while(True):
        
        que = input()#'saw a old yellow'
        if que =='exit':
            break
        else:
            que = preprocess_test(que, name_of_computer, x_tk)
        
        sentence =que
        
        sentence = [x_tk.word_index[word] for word in sentence.split()]
        

        sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
        sentences = np.array([sentence[0], x[0]])

        tmp_sentences = pad(sentences, y.shape[1])
        tmp_sentences = tmp_sentences.reshape((-1, y.shape[-2]))

        predictions = model.predict(tmp_sentences, len(tmp_sentences))

        print('Sample 1:')
        print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
        #print('Il a vu un vieux camion jaune')
        #print('Sample 2:')
        #print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
        #print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))

final_predictions(loaded_model,preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

computer: hi ! please type your name.

user: shyam
computer: hi , shyam ! My name is egg.

how are you
Sample 1:
konnichihuahua! <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Did you hear about the Native American man that drank 200 cups of tea?
Sample 1:
He pulled impeached <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
What's the best anti diarrheal prescription?
Sample 1:
mitosis. <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
How do you call it when an egg is on point?
Sample 1:
konnichihuahua! <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
exit


In [21]:
from keras.models import load_model

loaded_model = load_model('saved_models/final_rnn_model6.hdf5')
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 15, 128)           346496    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 15, 64)            41216     
_________________________________________________________________
dropout_9 (Dropout)          (None, 15, 64)            0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 15, 64)            24832     
_________________________________________________________________
dropout_10 (Dropout)         (None, 15, 64)            0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 15, 64)            24832     
_________________________________________________________________
dropout_11 (Dropout)         (None, 15, 64)            0         
__________

# Old model

In [53]:

def model_final6(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print(input_shape[1:])
    print(output_sequence_length)
    
    learning_rate = 0.01
    hidden_dim = 256
    embedding_vector_length = 100
    
    #Variation - 3 Hidden LSTM Layer and 2 Dense ouput layer with higher Dropout Rate
    
    model = Sequential()
    model.add(Embedding(english_vocab_size, embedding_vector_length, weights=encoder_embedding_matrix,trainable= False,
                        input_shape= input_shape[1:]))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Dense(french_vocab_size))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(french_vocab_size))
    model.add(Activation('softmax'))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    print(model.summary())
    return model


#tests.test_model_final(model_final)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
final_rnn_model6 = model_final6(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
final_rnn_model6.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=500, validation_split=0.2)

final_rnn_model6.save('./saved_models/final_rnn_model6_embed.hdf5')

# Print prediction(s)
print(logits_to_text(final_rnn_model6.predict(tmp_x[:1])[0], french_tokenizer))

print('Final Model Loaded')

(15,)
15


ValueError: You called `set_weights(weights)` on layer "embedding_6" with a  weight list of length 2707, but the layer was expecting 1 weights. Provided weights: [[ 0.          0.          0.         ...,  0.    ...

In [29]:
model.summary()

NameError: name 'model' is not defined

In [40]:
print(logits_to_text(model.predict(tmp_x[25:30])[0], french_tokenizer))


It <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [ ]:
#preproc_french_sentences[6]

len(french_tokenizer.word_index)+1
#print(logits_to_text(final_rnn_model6.predict(tmp_x[20]), french_tokenizer))


## Prediction (IMPLEMENTATION)

spacy.tokens.doc.Doc

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.